In [261]:
from email.headerregistry import AddressHeader
from cv2 import kmeans
import numpy as np
import json
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import mpl_toolkits.mplot3d as m3d
import matplotlib.pyplot as plt
from numpy import unique
from numpy import where
import math


class WavefrontGroup:
    def __init__(self, name='default'):
        self.name = name               # group name
        # vertices as an Nx3 or Nx6 array (per vtx colors)
        self.vertices = []
        self.normals = []                 # normals
        self.texcoords = []                 # texture coordinates
        # M*Nv*3 array, Nv=# of vertices, stored as vid,tid,nid (-1 for N/A)
        self.polygons = []


class WavefrontOBJ:
    def __init__(self, default_mtl='default_mtl'):
        self.path = None               # path of loaded object
        self.mtllibs = []                 # .mtl files references via mtllib
        self.mtls = [default_mtl]    # materials referenced
        self.mtlid = []                 # indices into self.mtls for each polygon
        # vertices as an Nx3 or Nx6 array (per vtx colors)
        self.vertices = []
        self.normals = []                 # normals
        self.texcoords = []                 # texture coordinates
        # M*Nv*3 array, Nv=# of vertices, stored as vid,tid,nid (-1 for N/A)
        self.polygons = []
        self.groups = []                 # Groups


def load_obj(filename: str, default_mtl='default_mtl', triangulate=False) -> WavefrontOBJ:
    """Reads a .obj file from disk and returns a WavefrontOBJ instance

    Handles only very rudimentary reading and contains no error handling!

    Does not handle:
        - relative indexing
        - subobjects or groups
        - lines, splines, beziers, etc.
    """
    # parses a vertex record as either vid, vid/tid, vid//nid or vid/tid/nid
    # and returns a 3-tuple where unparsed values are replaced with -1
    def parse_vertex(vstr):
        vals = vstr.split('/')
        vid = int(vals[0])-1
        tid = int(vals[1])-1 if len(vals) > 1 and vals[1] else -1
        nid = int(vals[2])-1 if len(vals) > 2 else -1
        return (vid, tid, nid)

    with open(filename, 'r') as objf:
        obj = WavefrontOBJ(default_mtl=default_mtl)
        obj.path = filename
        cur_mat = obj.mtls.index(default_mtl)
        cur_group = WavefrontGroup()

        for line in objf:
            toks = line.split()
            if not toks:
                continue
            if toks[0] == 'g':
                cur_group = WavefrontGroup(name=toks[1])
                obj.groups.append(cur_group)
            if toks[0] == 'v':
                cur_group.vertices.append([float(v) for v in toks[1:]])
            elif toks[0] == 'vn':
                cur_group.normals.append([float(v) for v in toks[1:]])
            elif toks[0] == 'vt':
                cur_group.texcoords.append([float(v) for v in toks[1:]])
            elif toks[0] == 'f':
                poly = [parse_vertex(vstr) for vstr in toks[1:]]
                if triangulate:
                    for i in range(2, len(poly)):
                        obj.mtlid.append(cur_mat)
                        cur_group.polygons.append(
                            (poly[0], poly[i-1], poly[i]))
                else:
                    obj.mtlid.append(cur_mat)
                    cur_group.polygons.append(poly)
            elif toks[0] == 'mtllib':
                obj.mtllibs.append(toks[1])
            elif toks[0] == 'usemtl':
                if toks[1] not in obj.mtls:
                    obj.mtls.append(toks[1])
                cur_mat = obj.mtls.index(toks[1])
        return obj


In [262]:
def PCA(data):
    # Calculate the mean of the points, i.e. the 'center' of the cloud
    datamean = data.mean(axis=0)
    print('mean ' + str(datamean))

    # PCA to generate a line representation for each tube
    mu = data.mean(0)
    C = np.cov(data - mu, rowvar=False)
    d, u = np.linalg.eigh(C)
    U = u.T[::-1]

    # Project points onto the principle axes
    Z = np.dot(data - mu, U.T)
    print('min Z ' + str(Z.min()))
    print('max Z ' + str(Z.max()))

    return Z, U, mu


In [263]:
def cluster(data):
    model = DBSCAN(eps=0.05, min_samples=9)

    # fit model and predict clusters
    labels = model.fit_predict(data)

    # retrieve unique clusters
    clusters = unique(labels)

    if False:
        x2 = Z.T[0]
        y2 = Z.T[1]

        # create scatter plot for samples from each cluster
        for cluster in clusters:
            # get row indexes for samples with this cluster
            row_ix = where(labels == cluster)
            # create scatter of these samples
            plt.scatter(x2[row_ix], y2[row_ix], s=3)
        # show the plot
        plt.rcParams['figure.figsize'] = [25, 5]
        plt.axis('scaled')
        plt.show()

    # print('labels ' + str(labels))
    # print('clusters ' + str(clusters))
    print('# of clusters ' + str(clusters.size))

    return clusters, labels


In [264]:
class Node:
    def __init__(self, p):
        self.p = p
        self.next = None

    def distance(self, other_node):
        return np.linalg.norm(self.p - other_node.p)


def createLineSegments(data, clusters, labels):
    # First create an unordered list of nodes, one per cluster with using the mean of the cluster points.
    nodes = []
    for cluster in clusters:
        row_ix = where(labels == cluster)
        cluster_points = data[row_ix]
        node = Node(p=cluster_points.mean(axis=0))
        nodes.append(node)

    # Connect nodes to line segments starting from a random node and extending in both directions the result is a start and an end node connected in a linked list
    start = nodes.pop()
    end = start
    while len(nodes) > 0:
        min_node = None
        min_dist = float('inf')
        closest_to_start = True

        for node in nodes:
            start_dist = start.distance(node)
            end_dist = end.distance(node)

            if start_dist < min_dist and start_dist <= end_dist:
                min_dist = start_dist
                min_node = node
                closest_to_start = True

            if end_dist < min_dist and end_dist < start_dist:
                min_dist = end_dist
                min_node = node
                closest_to_start = False

        if closest_to_start:
            min_node.next = start
            start = min_node
        else:
            end.next = min_node
            end = min_node

        nodes.remove(min_node)

    return start


def lineSegmentsLength(start):
    n = start
    total_length = 0
    while True:
        if n.next == None:
            break
        total_length = total_length + n.distance(n.next)
        n = n.next
    return total_length

# Generate LED positions by tracing along the line segments until the correct number of LED are created
def traceLineSegments(start, num_leds, offset, led_dist):
    segment_points = []
    fraction = 0
    n = start
    dist = offset
    for i in range(num_leds):
        new_fraction = fraction + dist / n.distance(n.next)
        while new_fraction >= 1:
            # Move to next segment
            dist = dist - (1 - fraction) * n.distance(n.next)
            n = n.next
            fraction = 0
            new_fraction = dist / n.distance(n.next)

        fraction = new_fraction
        p = n.p + fraction * (n.next.p - n.p)
        segment_points.append(p)
        dist = led_dist

    return np.array(segment_points)

def generateLEDPositions(data, clusters, labels):
    LEDS_PER_METER = 60  # eg. 300LEDs/16.4ft
    LED_DIST = 1 / LEDS_PER_METER
    LED_START_OFFSET = 0.1  # 10cm offset from that start of a segment
    LED_END_OFFSET = 0.1

    segments = createLineSegments(data, clusters, labels)
    length = lineSegmentsLength(segments)
    length = length - LED_START_OFFSET - LED_END_OFFSET
    num_leds = math.floor(length * LEDS_PER_METER)
    points = traceLineSegments(segments, num_leds, LED_START_OFFSET, LED_DIST)
    return points, length


In [265]:
def prepareData(group):
    x = np.array([v[0] for v in group.vertices])
    y = np.array([v[1] for v in group.vertices])
    z = np.array([v[2] for v in group.vertices])

    data = np.concatenate((x[:, np.newaxis],
                        y[:, np.newaxis],
                        z[:, np.newaxis]),
                        axis=1)
    return data

In [266]:
def plotSegment(data, clusters, labels, segment_points):
    Z, U, mu = PCA(data)
    x2 = Z.T[0]
    y2 = Z.T[1]

    # create scatter plot for samples from each cluster
    for cluster in clusters:
        # get row indexes for samples with this cluster
        row_ix = where(labels == cluster)
        # create scatter of these samples
        plt.scatter(x2[row_ix], y2[row_ix], s=3)

    # create scatter plot for line segments
    L = np.dot(segment_points - mu, U.T)
    xl2 = L.T[0]
    yl2 = L.T[1]
    plt.scatter(xl2, yl2, s=3)

    # show the plot
    plt.rcParams['figure.figsize'] = [25, 5]
    plt.axis('scaled')
    plt.show()


In [275]:
class Segment():
    def __init__(self, name, points, length):
        self.name = name
        self.points = points
        self.num_leds = points.shape[0]
        self.length = length

scene = load_obj('led_components.obj')
segments = []
for group in scene.groups:
    print('Processing: ' + group.name)
    data = prepareData(group)
    clusters, labels = cluster(data)
    points, length = generateLEDPositions(data, clusters, labels)
    segments.append(Segment(group.name, points, length))
    # plotSegment(data, clusters, labels, segment.points)

Processing: Body1
# of clusters 4
Processing: Body2
# of clusters 7
Processing: Body3
# of clusters 9
Processing: Body4
# of clusters 9
Processing: Body5
# of clusters 9
Processing: Body6
# of clusters 4
Processing: Body7
# of clusters 7
Processing: Body8
# of clusters 5
Processing: Body9
# of clusters 9
Processing: Body10
# of clusters 9
Processing: Body11
# of clusters 5
Processing: Body12
# of clusters 9
Processing: Body13
# of clusters 5
Processing: Body14
# of clusters 5
Processing: Body15
# of clusters 5
Processing: Body16
# of clusters 9
Processing: Body17
# of clusters 7
Processing: Body18
# of clusters 9
Processing: Body19
# of clusters 4
Processing: Body20
# of clusters 9
Processing: Body21
# of clusters 9
Processing: Body22
# of clusters 5
Processing: Body23
# of clusters 5
Processing: Body24
# of clusters 7
Processing: Body25
# of clusters 9
Processing: Body26
# of clusters 5
Processing: Body27
# of clusters 5
Processing: Body28
# of clusters 9
Processing: Body29
# of clust

In [276]:
uid = 1
json_dict = {
    'total_num_leds': 0,
    'total_length': 0,
    'led_segments': []
}

total_num_leds = 0
total_length = 0
for segment in segments:
    total_num_leds = total_num_leds + segment.num_leds
    total_length = total_length + segment.length
json_dict['total_num_leds'] = total_num_leds
json_dict['total_length'] = total_length

for segment in segments:
    json_dict['led_segments'].append(
        {
            'uid': uid,
            'name': segment.name,
            'num_leds': segment.num_leds,
            'length': segment.length,
            'led_positions': segment.points.tolist()
        })
    uid = uid + 1

with open('led_config.json', 'w', encoding='utf-8') as f:
    json.dump(json_dict, f, ensure_ascii=False, indent=4)
